In [4]:
import matplotlib.pyplot as plt


ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
ax1.set_adjustable('box-forced')
plt.show()

In [ ]:
#the first thing is to check that we can create a morlet filter and apply it to an image

In [ ]:
def gabor_2d(shape, sigma0, xi, theta, slant=None, pyramid=False, offset=None):
    """Returns a Gabor filter following the specifications

    This function creates a 2D complex Gabor filter. All parameters
    are taken to be in integer grid space


    Parameters
    ----------
    shape : {tuple, list}
        shape=(2,)
        Indicates the shape of the output array
    sigma_0 : float
        Indicates the standard deviation of the filter envelope along
        the oscillation axis
    slant : float
        Indicates the standard deviation of the filter envelope along
        the axis orthogonal to the oscillation. It is given relatively
        with respect to sigma_0 (sigma_orthog = slant * sigma_0)
    xi : float
        The oscillation wave number
    theta : float
        The oscillation wave orientation 
        (0 is a downward pointing wave vector)
    offset : {tuple, list, ndarray}
        shape=(2,)
        Possible offset for the filter from the origin. Defaults to 0


    See also
    --------
    Morlet wavelets

    """

    # to be compatible with the Matlab version
    shape = (shape[1], shape[0])

    if slant is None:
        slant = 1.
    if offset is None:
        if pyramid:
            offset = (shape[0]/2, shape[1]/2)
        else:
            offset = (-(-shape[0]/2), -(-shape[1]/2))

    offset = np.asanyarray(offset).reshape(2, 1, 1)

    # to understand the following, note that 5 / 2 = 2, -5 / 2 = -3
    g = np.mgrid[0:shape[0], 0:shape[1]]
    g -= offset
    print g[:3, :3, :]

    rot = rotation_matrix_2d(theta)
    invrot = np.linalg.inv(rot)
    rot_g = invrot.dot(g.reshape(2, -1))
    precision_matrix = np.diag([1., slant ** 2]) / (sigma0 ** 2)
    mahalanobis = (rot_g * (precision_matrix.dot(rot_g))).sum(0)

    raw_gabor = np.exp(-mahalanobis / 2 + 1j * xi * rot_g[0])

    if pyramid:
        gabor = raw_gabor.reshape(shape) / (2 * np.pi * sigma0**2 / slant**2)
    else:
        gabor = np.fft.fftshift(raw_gabor.reshape(shape)) / (
            2 * np.pi * sigma0**2 / slant**2)

    # return transposed in order to be compatible with matlab version
    return gabor.T

In [6]:
def morlet_2d(sigma, xi, theta, slant=None):
#spatial domain (non-Fourier)
    gabor = gabor_2d(sigma, xi, theta, slant)
    envelope = np.abs(gabor)
    K = gabor.sum() / envelope.sum()

    centered = gabor - K * envelope

    return centered


(8192,)